In [ ]:
import torch
import cv2
import torchvision
import numpy as np
from torchvision import transforms
from google.colab.patches import cv2_imshow  # Use cv2.imshow() if running locally

# Define COCO class names (from index 1 to 91)
class_names = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "street sign", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
    "hat", "backpack", "umbrella", "shoe", "eye glasses", "handbag", "tie", "suitcase",
    "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "plate", "wine glass", "cup", "fork",
    "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot",
    "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "mirror",
    "dining table", "window", "desk", "toilet", "door", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "blender",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush", "hair brush"
]

# Load Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Image preprocessing function
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert to tensor (scales to 0-1 automatically)
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Load image
image_path = "/content/jason-briscoe-GliaHAJ3_5A-unsplash.jpg"  # Change path accordingly
image = cv2.imread(image_path)

if image is None:
    raise ValueError(f"Error: Image '{image_path}' not found!")

# Convert BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Preprocess image for PyTorch model
preprocessed_image = preprocess_image(image_rgb)

# Perform inference
with torch.no_grad():
    detections = model(preprocessed_image)

# Extract bounding boxes, labels, and scores
boxes = detections[0]['boxes'].cpu().numpy()
labels = detections[0]['labels'].cpu().numpy()
scores = detections[0]['scores'].cpu().numpy()

# Set a confidence threshold
conf_threshold = 0.69  # Adjust as needed
high_conf_indices = np.where(scores > conf_threshold)[0]

if len(high_conf_indices) == 0:
    print("No objects detected with confidence > 0.69.")
else:
    print(f"Detected {len(high_conf_indices)} objects with confidence > {conf_threshold}")

# Draw bounding boxes on image
for i in high_conf_indices:
    x1, y1, x2, y2 = map(int, boxes[i])
    label = labels[i]

    # Fix label index (COCO labels start from 1)
    if label > 0 and label <= len(class_names):
        class_name = class_names[label - 1]  # Correcting zero-index issue
    else:
        class_name = f"Unknown ({label})"

    # Draw bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, class_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Display the image
cv2_imshow(image)  # Use cv2.imshow() if running locally
cv2.waitKey(0)
cv2.destroyAllWindows()
